In [1]:
%%bash

pip install pytorch-lightning torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
import functools

import pytorch_lightning as pl
from sklearn import datasets as skdatasets 
import torch
from torch import nn, optim, utils
import torchinfo
import torchmetrics
import torchvision
from torchvision import datasets as tvdatasets
from torchvision import models, transforms

## Implementing Convolutional Layers in PyTorch

In [ ]:
nn.Conv1d?

In [25]:
nn.Conv2d?

In [ ]:
nn.Conv3d?

In [3]:
samples = skdatasets.load_sample_images()

In [41]:
torch.from_numpy?

In [4]:
image_tensors = torch.stack([torch.from_numpy(img / 255).float() for img in samples["images"]], dim=0)

In [5]:
image_tensors.shape

torch.Size([2, 427, 640, 3])

In [6]:
image_tensors.dtype

torch.float32

In [7]:
conv_2d = nn.Conv2d(
    in_channels=3,
    out_channels=32,
    kernel_size=7,
    stride=1,
)

In [46]:
conv_2d(image_tensors)

RuntimeError: ignored

In [8]:
output = conv_2d(image_tensors.permute(0, 3, 1, 2))

In [9]:
output.shape

torch.Size([2, 32, 421, 634])

In [11]:
conv_2d = nn.Conv2d(
    in_channels=3,
    out_channels=32,
    kernel_size=7,
    stride=1,
    padding="same",
)

In [12]:
output = conv_2d(image_tensors.permute(0, 3, 1, 2))

In [13]:
output.shape

torch.Size([2, 32, 427, 640])

## Implemententing pooling layers using PyTorch

In [14]:
nn.MaxPool2d?

In [16]:
max_pool2d = nn.MaxPool2d(
    kernel_size=(2, 2),
)

In [17]:
pooling_output = max_pool2d(output)

In [18]:
pooling_output.shape

torch.Size([2, 32, 213, 320])

In [19]:
pooling_output

tensor([[[[ 7.2376e-02,  2.1506e-01,  2.5275e-02,  ...,  9.1908e-02,
            9.2018e-02,  4.2687e-02],
          [-5.4077e-02,  1.1178e-01, -1.3067e-01,  ..., -1.0762e-01,
           -1.0695e-01,  3.0905e-02],
          [-3.0231e-01, -1.3830e-01, -3.6894e-01,  ..., -4.1644e-01,
           -3.8630e-01, -3.9329e-02],
          ...,
          [-2.3289e-02,  1.4141e-01,  4.9261e-02,  ...,  5.8869e-02,
            4.6144e-02,  5.0668e-02],
          [-2.2873e-02,  1.6420e-01, -4.4328e-02,  ...,  9.1813e-02,
            1.0529e-01,  8.5742e-02],
          [ 1.6500e-03,  7.8455e-02, -2.4727e-02,  ...,  5.6867e-03,
            6.9745e-02,  7.3705e-02]],

         [[ 4.1219e-01,  5.3866e-01,  4.7820e-01,  ...,  5.2357e-01,
            7.3557e-01,  7.7411e-01],
          [ 5.1151e-01,  7.3702e-01,  6.1543e-01,  ...,  6.7340e-01,
            8.2792e-01,  1.0369e+00],
          [ 5.1014e-01,  7.3778e-01,  6.1733e-01,  ...,  6.7640e-01,
            8.3277e-01,  1.0385e+00],
          ...,
     

In [ ]:
nn.AvgPool2d?

In [21]:
avg_pool2d = nn.AvgPool2d(
    kernel_size=(2,2)
)

In [22]:
pooling_output = avg_pool2d(output)

In [23]:
pooling_output.shape

torch.Size([2, 32, 213, 320])

In [24]:
pooling_output

tensor([[[[ 3.4042e-02,  9.2201e-02, -1.6822e-02,  ...,  4.4551e-02,
           -2.6486e-03, -5.6350e-02],
          [-2.6518e-01, -1.3307e-01, -2.5182e-01,  ..., -2.6361e-01,
           -2.8184e-01, -4.1560e-02],
          [-3.9794e-01, -2.5465e-01, -3.7131e-01,  ..., -4.1697e-01,
           -4.0180e-01, -8.0626e-02],
          ...,
          [-9.5800e-02,  7.5253e-02,  1.1992e-02,  ...,  4.6022e-02,
            2.5520e-02,  3.1696e-02],
          [-1.0466e-01,  5.4003e-02, -5.9730e-02,  ...,  4.9257e-02,
            7.5727e-02,  5.8748e-02],
          [-1.3883e-01,  3.8080e-02, -5.0989e-02,  ..., -1.3622e-02,
            4.8583e-02,  5.6620e-02]],

         [[ 2.2253e-01,  4.3096e-01,  4.1515e-01,  ...,  4.4072e-01,
            5.6786e-01,  7.1584e-01],
          [ 2.3861e-01,  5.7364e-01,  5.4233e-01,  ...,  5.7510e-01,
            6.7484e-01,  9.5485e-01],
          [ 3.1128e-01,  6.7648e-01,  6.1560e-01,  ...,  6.7449e-01,
            7.5338e-01,  1.0215e+00],
          ...,
     

# Implementing a Deep CNN using PyTorch

## Loading and Preprocessing the Data

In [70]:
class MNISTDataModule(pl.LightningDataModule):

    def __init__(self, data_dir, batch_size):
        super().__init__()

        self.batch_size = batch_size
        self.data_dir = data_dir
        self.dims = (1, 28, 28)
        self.num_classes = 10
        self.transform = transforms.Compose(
            [
                transforms.RandomRotation(degrees=15),
                transforms.ToTensor(),
            ]
        )

    def prepare_data(self):

        _ = tvdatasets.MNIST(
            self.data_dir,
            download=True,
            train=True,
        )

        _ = tvdatasets.MNIST(
            self.data_dir,
            download=True,
            train=False,
        )

    def setup(self, stage=None):

        # assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            _train_dataset = tvdatasets.MNIST(
                self.data_dir,
                train=True,
                transform=self.transform,
                download=False
            )
            self.train, self.val = utils.data.random_split(
                _train_dataset,
                lengths = [48000, 12000]
            )

        # assign test dataset for use in dataloader
        if stage == "test" or stage is None:
            self.test = tvdatasets.MNIST(
                self.data_dir,
                train=False,
                transform=self.transform,
                download=False
            )

    def train_dataloader(self):
        return utils.data.DataLoader(
            self.train,
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return utils.data.DataLoader(
            self.val,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def test_dataloader(self):
        return utils.data.DataLoader(
            self.test,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def predict_dataloader(self):
        return utils.data.DataLoader(
            self.test,
            batch_size=self.batch_size,
            shuffle=False,
        )


## Typical CNN implementation

In [81]:
class LightningClassifier(pl.LightningModule):

    def __init__(self, lr=1e-2, momentum=0, nesterov=False):
        super().__init__()

        self.model_fn = nn.Sequential(
            *self.make_conv2d_layer(1, 64, (7, 7)),
            nn.MaxPool2d(kernel_size=(2, 2)),
            *self.make_conv2d_layer(64, 128),
            *self.make_conv2d_layer(128, 128),
            nn.MaxPool2d(kernel_size=(2, 2)),
            *self.make_conv2d_layer(128, 256),
            *self.make_conv2d_layer(256, 256),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Flatten(),
            *self.make_linear_layer(1024, 128),
            nn.Dropout(0.5),
            *self.make_linear_layer(128, 64),
            nn.Dropout(0.5),
            nn.Linear(64, 10),
            nn.Softmax(dim=1)
        )

        self.loss_fn = nn.NLLLoss()

        self.train_accuracy = (torchmetrics.classification
                                           .MulticlassAccuracy(10))
        self.val_accuracy = (torchmetrics.classification
                                         .MulticlassAccuracy(10))
        self.test_accuracy = (torchmetrics.classification
                                          .MulticlassAccuracy(10))

        self.lr = lr
        self.momentum=momentum
        self.nesterov=nesterov

    @staticmethod
    def make_conv2d_layer(in_channels, out_channels, kernel_size=(3, 3)):
        conv2d = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            padding="same",
        )
        nn.init.kaiming_normal_(conv2d.weight)
        nn.init.constant_(conv2d.bias, 0.0)
        return conv2d, nn.ReLU()

    @staticmethod
    def make_linear_layer(in_features, out_features):
        linear = nn.Linear(in_features, out_features)
        nn.init.kaiming_normal_(linear.weight)
        nn.init.constant_(linear.bias, 0.0)
        return linear, nn.ReLU()

    def forward(self, X):
        output = self.model_fn(X)
        return output

    def training_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.val_accuracy(logits, y)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.val_accuracy(logits, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.test_accuracy(logits, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        X, _ = batch
        logits = self.forward(X)
        y_hat = torch.argmax(logits, dim=1)
        return y_hat

    def configure_optimizers(self):
        _optimizer_hyperparameters = {
            "lr": self.lr,
            "momentum": self.momentum,
            "nesterov": self.nesterov,
        }
        optimizer = optim.SGD(
            self.parameters(),
            **_optimizer_hyperparameters
        )
        return optimizer



In [82]:
_optimizer_hyperparameters = {
    "momentum": 0.0,
    "nesterov": False,
}
classifier = LightningClassifier(**_optimizer_hyperparameters)

_trainer_kwargs = {
    "callbacks": [
        pl.callbacks.early_stopping.EarlyStopping("val_loss", mode="min"),
    ],
    "max_epochs": 100
}
trainer = pl.Trainer(**_trainer_kwargs)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [83]:
torchinfo.summary(classifier, input_shape=(32, 1, 28, 28))

Layer (type:depth-idx)                   Param #
LightningClassifier                      --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       3,200
│    └─ReLU: 2-2                         --
│    └─MaxPool2d: 2-3                    --
│    └─Conv2d: 2-4                       73,856
│    └─ReLU: 2-5                         --
│    └─Conv2d: 2-6                       147,584
│    └─ReLU: 2-7                         --
│    └─MaxPool2d: 2-8                    --
│    └─Conv2d: 2-9                       295,168
│    └─ReLU: 2-10                        --
│    └─Conv2d: 2-11                      590,080
│    └─ReLU: 2-12                        --
│    └─MaxPool2d: 2-13                   --
│    └─Flatten: 2-14                     --
│    └─Linear: 2-15                      131,200
│    └─ReLU: 2-16                        --
│    └─Dropout: 2-17                     --
│    └─Linear: 2-18                      8,256
│    └─ReLU: 2-19                        

In [84]:
datamodule = MNISTDataModule("./sample_data", batch_size=32)

In [85]:
trainer.fit(
    model=classifier,
    datamodule=datamodule,
)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | model_fn | Sequential         | 1.2 M 
1 | loss_fn  | NLLLoss            | 0     
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
5.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### LeNet Architecture

### AlexNet

In [ ]:
nn.AdaptiveAvgPool2d?

In [ ]:
nn.LocalResponseNorm?

In [ ]:
models.alexnet?

### GoogLeNet

In [ ]:
models.googlenet?

In [ ]:
models.inception_v3?

### VGGNet

In [42]:
models.vgg11?

### ResNet

In [ ]:
models.resnet50?

### ResNeXt

In [44]:
models.resnext50_32x4d?

### EfficientNet

In [46]:
models.efficientnet_v2_m?

### Coding a ResNet from scratch using PyTorch


In [97]:
class ResidualUnit(nn.Module):

    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        self.main_layers = [
            self.make_conv2d_layer(in_channels, out_channels, stride=stride),
            nn.BatchNorm2d(),
            nn.ReLU(),
            self.make_conv2d_layer(out_channels, out_channels, stride=stride),
            nn.BatchNorm2d(),
        ]

        if stride > 1:
            self.skip_layers = [
                self.make_conv2d_layer(
                    in_channels,
                    out_channels,
                    kernel_size=(1, 1),
                    stride=stride
                ),
                nn.BatchNorm2d()
            ]
        else:
            self.skip_layers = []

    @staticmethod
    def make_conv2d_layer(in_channels, out_channels, kernel_size=(3, 3), stride=1):
        conv2d = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding="same",
            bais=False,
        )
        nn.init.kaiming_normal_(conv2d.weight)
        nn.init.constant_(conv2d.bias, 0.0)
        return conv2d

    def forward(self, X):
        Z0 = functools.reduce(lambda Z, f: f(Z), self.main_layers, X)
        Z1 = functools.reduce(lambda Z, f: f(Z), self.skip_layers, X)
        return nn.ReLU()(Z0 + Z1)


class LightningClassifier(pl.LightningModule):

    def __init__(self, lr=1e-2, momentum=0, nesterov=False):
        super().__init__()

        model = nn.Sequential(
            self.make_conv2d_layer(3, 64, kernel_size=(7, 7), stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding="same"),
        )

        in_channels = 64
        for out_channels in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
            stride = 1 if out_channels == in_channels else 2
            model.add(ResidualUnit(in_channels, out_channels, stride=stride))
            in_channels = out_channels

        model.add(nn.AdaptiveAvgPool2d(output_size=(1, 1)))
        model.add(nn.Flatten())
        model.add(self.make_linear_layer(512, 10))
        model.add(nn.Softmax(dim=1))

        self.loss_fn = nn.NLLLoss()

        self.train_accuracy = (torchmetrics.classification
                                           .MulticlassAccuracy(10))
        self.val_accuracy = (torchmetrics.classification
                                         .MulticlassAccuracy(10))
        self.test_accuracy = (torchmetrics.classification
                                          .MulticlassAccuracy(10))

        self.lr = lr
        self.momentum=momentum
        self.nesterov=nesterov

    @staticmethod
    def make_conv2d_layer(in_channels, out_channels, kernel_size=(3, 3)):
        conv2d = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            padding="same",
        )
        nn.init.kaiming_normal_(conv2d.weight)
        nn.init.constant_(conv2d.bias, 0.0)
        return conv2d

    @staticmethod
    def make_linear_layer(in_features, out_features):
        linear = nn.Linear(in_features, out_features)
        nn.init.kaiming_normal_(linear.weight)
        nn.init.constant_(linear.bias, 0.0)
        return linear

    def forward(self, X):
        output = self.model_fn(X)
        return output

    def training_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.val_accuracy(logits, y)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.val_accuracy(logits, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.test_accuracy(logits, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        X, _ = batch
        logits = self.forward(X)
        y_hat = torch.argmax(logits, dim=1)
        return y_hat

    def configure_optimizers(self):
        _optimizer_hyperparameters = {
            "lr": self.lr,
            "momentum": self.momentum,
            "nesterov": self.nesterov,
        }
        optimizer = optim.SGD(
            self.parameters(),
            **_optimizer_hyperparameters
        )
        return optimizer



In [ ]:
models.

# Transfer Learning with PyTorch

In [89]:
model = models.resnet50(weights=???)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
class LightningClassifier(pl.LightningModule):

    def __init__(self, lr=1e-2, momentum=0, nesterov=False):
        super().__init__()

        self.model_fn = models.resnet50(pretrained=True)

        self.loss_fn = nn.NLLLoss()

        self.accuracy = torchmetrics.classification.MulticlassAccuracy(10)

        self.lr = lr

        self.momentum=momentum

        self.nesterov=nesterov

    def forward(self, X):
        output = self.model_fn(X)
        return output

    def training_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.accuracy(logits, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        X, y = batch
        logits = self.forward(X)
        loss = self.loss_fn(logits, y)
        acc = self.accuracy(logits, y)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        X, _ = batch
        logits = self.forward(X)
        y_hat = torch.argmax(logits, dim=1)
        return y_hat

    def configure_optimizers(self):
        _optimizer_hyperparameters = {
            "lr": self.lr,
            "momentum": self.momentum,
            "nesterov": self.nesterov,
        }
        optimizer = optim.SGD(
            self.parameters(),
            **_optimizer_hyperparameters
        )
        return optimizer

